# Reference variables
1. Variables in Python are **not boxes** (as opposed to variables in C/C++) but mere labels bond to objects.
    - Statement `x = ...` binds the name `x` to the object created/referenced on the righthand side.
2. Multiple variables can be bond to an object, in which these variables are indeed aliases.

# is vs ==
1. `is` compares identities of objects (i.e. in CPython, objects' memory addresses), while `==` compares values of objects.
    - An object's identity never changes after it's created.
2. <font color='red'>**Performance consideration**</font>: `is` is faster than `==` because it cannot be overloaded so that Python does not have to find and invoke special methods to evaluate it (unlike in `==` where the `__eq__` method would be called).
3. `is None` is the only common use case, while `==` is usually what we want.


In [5]:
alex = {'name': 'alex', 'age': 21}
jane = alex  # jane is an alias for alex

assert alex == jane
assert alex is jane  # they refer to the same object
print(id(alex), id(jane))

tom = {'name': 'alex', 'age': 21}
assert tom == alex  # values are equal
assert tom is not alex  # but they do not refer to the same object
print(id(tom), id(alex))

140208027566912 140208027566912
140208027613184 140208027566912


# Copies are shallow by default
1. `l2 = list(l1)` or `l2 = l1[:]` produces a shallow copy.
    - The outermost container is duplicated (i.e. they are distinct objects).
    - Objects in the new container are still references to the same items in the original container.
2. Shallow copies may bring surprices when container elements are **mutable**.
3. Use `copy` module's `deepcopy` for deep copy or `copy` for shallow copy.
4.

In [15]:
l1 = [1, [2, 3], (4, 5)]
l2 = list(l1)  # l2 is a shallow copy of l1
assert l1 == l2
assert l1 is not l2

l1.append(6)
assert l1 == [1, [2, 3], (4, 5), 6]
assert l2 == [1, [2, 3], (4, 5)]  # l2 is not affected

l2[1] += [-1]
assert l2 == [1, [2, 3, -1], (4, 5)]
assert l1 == [1, [2, 3, -1], (4, 5), 6]  # l1 is affected

l2[2] += (-2, )
assert l2 == [1, [2, 3, -1], (4, 5, -2)]
assert l1 == [1, [2, 3, -1], (4, 5), 6]  # l1 is not affected

## Visualization of the above example
![Image](./asset/c6_shallow_copy.png)

In [20]:
class Bus:
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


import copy


bus1 = Bus([1, 2, 3, 4])
bus2 = copy.copy(bus1)  # bus2 is a shallow copy of bus1 -> they share the passenger list object
bus3 = copy.deepcopy(bus1)

print(id(bus1), id(bus2), id(bus3))
assert (bus1 is not bus2) and (bus1 is not bus3)

bus1.drop(1)
assert bus1.passengers == [2, 3, 4]
assert bus2.passengers == [2, 3, 4]  # bus2 passengers list is also changed
assert bus3.passengers == [1, 2, 3, 4]

140208029871072 140208022355344 140208022352752


## Visualization of the above example
![Image](./asset/c6_shallow_copy_objects.png)

# Function parameters as references

1. Parameter passing in Python is **call by sharing** (i.e. a reference is passed rather than the actual value).
2. When the passed parameter is mutable, there might be surprising effects.
3. Function developer should consider if the function caller would expect the passed parameters to be changed (if the passed parameters are mutable).
4. Avoid using mutable type as default function parameter
    - The default values are evaluted when the function is defined (i.e. usually when the module is laoded) and the default values becomes **attributes of the function object**.
    - If a default value is mutable, when it is changed, the change will affect every future call of the function.

In [28]:
# example: pass parameter by reference
def f(a, b):
    a += b
    return a

x = 1
y = 2
result = f(x, y)
assert result == 3
assert x == 1  # int is immutable
assert y == 2

x = [1, 2]
y = [3, 4]
result = f(x, y)
assert result == [1, 2, 3, 4]
assert x == [1, 2, 3, 4]
assert y == [3, 4]

x = (1, 2)
y = (3, 4)
result = f(x, y)
assert result == (1, 2, 3, 4)
assert x == (1, 2)
assert y == (3, 4)

## Visualization of the above example

### List
![Image](./asset/c6_function_parameter.png)

### Tuple
![Image](./asset/c6_function_parameter_immutable.png)

In [29]:
# example: avoid using mutable default function parameters
class Bus:
    def __init__(self, passengers = []):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


bus1 = Bus()

bus1.pick(1)
assert bus1.passengers == [1]

bus2 = Bus()
assert bus2.passengers == [1]

## Visualization of the above example
![Image](./asset/c6_function_parameter_mutable_default.png)